In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 1536



In [19]:
# data
sas.submitLST(
f"""
proc sort data=sashelp.cars out=cars;
    by type, origin;

proc summary data=cars missing nway;
    by type;
    class origin;
    var msrp;
    output out=sdf (drop=) mean=mean;
run;

title;
proc print data=sdf (obs=5);
run;
""")

Obs,Type,Origin,_TYPE_,_FREQ_,mean
1,Hybrid,Asia,1,3,"$19,920"
2,SUV,Asia,1,25,"$29,569"
3,SUV,Europe,1,10,"$48,346"
4,SUV,USA,1,25,"$34,589"
5,Sedan,Asia,1,94,"$22,764"


In [27]:
# by vars not transposed
# id vars formatted values will be the columns, must be unique within the by group
# var vars will be transposed, multiple vars will be stacked
sas.submitLST(
'''
PROC TRANSPOSE DATA=sdf OUT=sdf2;
    BY type;
    id origin;
    VAR _freq_ mean;
RUN;

PROC PRINT DATA=sdf2;
RUN;
'''
)


Obs,Type,_NAME_,Asia,Europe,USA
1,Hybrid,_FREQ_,3.00,.,.
2,Hybrid,mean,19920.00,.,.
3,SUV,_FREQ_,25.00,10.00,25.00
4,SUV,mean,29569.00,48346.00,34589.20
5,Sedan,_FREQ_,94.00,78.00,90.00
6,Sedan,mean,22763.97,42992.05,25638.83
7,Sports,_FREQ_,17.00,23.00,9.00
8,Sports,mean,32510.65,71998.70,45257.22
9,Truck,_FREQ_,8.00,.,16.00
10,Truck,mean,20383.63,.,27220.25


In [31]:
# simple pivot
sas.submitLST(
'''
PROC TRANSPOSE DATA=sdf OUT=sdf2 (DROP=_NAME_);
    BY type;
    id origin;
    VAR mean;
RUN;

PROC PRINT DATA=sdf2;
RUN;
'''
)


Obs,Type,Asia,Europe,USA
1,Hybrid,"$19,920",.,.
2,SUV,"$29,569","$48,346","$34,589"
3,Sedan,"$22,764","$42,992","$25,639"
4,Sports,"$32,511","$71,999","$45,257"
5,Truck,"$20,384",.,"$27,220"
6,Wagon,"$23,144","$37,851","$22,346"


In [33]:
# simple pivot, compare to pandas
df = sas.sasdata("sdf","work").to_df()
df.pivot(index='Type', columns='Origin', values='mean')

Origin,Asia,Europe,USA
Type,,,
Hybrid,19920.000000,NaN,NaN
SUV,29569.000000,48346.000000,34589.200000
Sedan,22763.968085,42992.051282,25638.833333
Sports,32510.647059,71998.695652,45257.222222
Truck,20383.625000,NaN,27220.250000
Wagon,23143.727273,37851.250000,22345.714286
